In [1]:
import time
import re
import requests 
import numpy as np
import pandas as pd
import concurrent.futures

from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementNotInteractableException,NoSuchWindowException 

from selenium.webdriver.remote.remote_connection import LOGGER, logging
LOGGER.setLevel(logging.WARNING)

# Scraping prices of products off amazon

### Load excel from before

In [79]:
df = pd.read_excel('pricelist.xls')
df.head()

,Item,,Designer,Category,Item Name,Price,Qty Avail,Packaging,UPC/EAN,Case Count,Order Qty,Gender
0,12672,1.0,Abercrombie & Fitch,Regular,Abercrombie & Fitch Fierce 1.7 EDC Sp Men,46.5,56,New In Box,634349765,12,,Men
1,26413,1.0,Al Haramain,Regular,Al Haramain Amber Oud Tobacco Edition 2.0 EDP ...,38.5,more than 360,New In Box,6291100132171,- None -,,Men
2,10689,1.0,Alfred Sung,Regular,Alfred Sung Hei 3.4 EDT Sp Men,12.5,more than 360,New In Box,067724200017,24,,Men
3,13377,1.0,Alfred Sung,Regular,Alfred Sung Shi 3.4 EDP Sp Women,14.5,12,New In Box,067724271116,24,,Women
4,11668,1.0,Animale,Regular,Animale 3.4 EDP Sp Women,21.5,more than 360,New In Box,892456000037,24,,Women


In [80]:
df.columns

Index(['Item', ' ', 'Designer', 'Category', 'Item Name', 'Price', 'Qty Avail',
       'Packaging', 'UPC/EAN', 'Case Count', 'Order Qty', 'Gender'],
      dtype='object')

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1289 entries, 0 to 1288
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Item        1289 non-null   int64  
 1               1284 non-null   float64
 2   Designer    1289 non-null   object 
 3   Category    1289 non-null   object 
 4   Item Name   1289 non-null   object 
 5   Price       1289 non-null   float64
 6   Qty Avail   1289 non-null   object 
 7   Packaging   1289 non-null   object 
 8   UPC/EAN     1284 non-null   object 
 9   Case Count  1289 non-null   object 
 10  Order Qty   1289 non-null   object 
 11  Gender      1289 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 121.0+ KB


In [82]:
# Make an array of the UPC values we will use to retrieve items.
upc_arr = df['UPC/EAN'].loc[df['UPC/EAN'] != '- None -'].to_numpy()
upc_arr

array(['634349765', '6291100132171', '067724200017', ..., '888874001541',
       '860002058931', '605930779929'], dtype=object)

In [83]:
# Helper function to search for product
def make_url(search_term):
    """
    Makes an amazon.ae search result url
    """
    return f'https://www.amazon.ae/s?k={search_term}'

### Try with one item

In [7]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-agent=[user-agent string]")
chrome_options.add_argument('--disable-notifications')
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument("window-size=1920,1080")

driver_manager = ChromeDriverManager().install()
driver = webdriver.Chrome(driver_manager,
                          options = chrome_options)


url = 'https://www.amazon.ae/s?k=3386460036757'
driver.get(url)
soup = bs(driver.page_source, 'html.parser') 

driver.quit()



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Get LATEST driver version for 97.0.4692
Trying to download new driver from https://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Fardin\.wdm\drivers\chromedriver\win32\97.0.4692.71]


In [8]:
soup.find_all('span', {'class':'a-offscreen'})

[<span class="a-offscreen">AED 109.00</span>,
 <span class="a-offscreen">AED 184.38</span>,
 <span class="a-offscreen">AED 109.00</span>]

In [9]:
soup.find_all('span', {'class':'a-offscreen'})[0].text

'AED\xa0109.00'

In [10]:
def clean_price(price_text):
    """
    Cleans up the price text.
    """
    regex = r'[\d]+\.?[\d]+'
    
    return re.findall(regex,price_text)[0]

clean_price('AED\xa0184.38')

'184.38'

## Do with all items of interest

In [11]:
df

,Item,,Designer,Category,Item Name,Price,Qty Avail,Packaging,UPC/EAN,Case Count,Order Qty,Gender
0,12672,1.0,Abercrombie & Fitch,Regular,Abercrombie & Fitch Fierce 1.7 EDC Sp Men,46.5,56,New In Box,634349765,12,,Men
1,26413,1.0,Al Haramain,Regular,Al Haramain Amber Oud Tobacco Edition 2.0 EDP ...,38.5,more than 360,New In Box,6291100132171,- None -,,Men
2,10689,1.0,Alfred Sung,Regular,Alfred Sung Hei 3.4 EDT Sp Men,12.5,more than 360,New In Box,067724200017,24,,Men
3,13377,1.0,Alfred Sung,Regular,Alfred Sung Shi 3.4 EDP Sp Women,14.5,12,New In Box,067724271116,24,,Women
4,11668,1.0,Animale,Regular,Animale 3.4 EDP Sp Women,21.5,more than 360,New In Box,892456000037,24,,Women
...,...,...,...,...,...,...,...,...,...,...,...,...
1284,28529,NaN,Bond 9,HBA,Bond No. 9 Madison Square Park 6.4 Scented Candle,55.0,10,New In Box,888874002333,6,,Unisex
1285,28531,NaN,Bond 9,HBA,Bond No. 9 Nolita 6.4 Scented Candle,60.0,4,New In Box,888874005747,6,,Unisex
1286,28533,NaN,Bond 9,HBA,Bond No. 9 The Scent Of Peace 6.4 Scented Candle,55.0,6,New In Box,888874001541,2,,Unisex
1287,26724,NaN,Nuit D'Orient,HBA,Nuit D'Orient Hand Sanitizer 16.0 OZ. Pump,3.0,15,Unboxed,860002058931,24,,Unisex


In [12]:
df_reg = df.loc[df['Category'] == 'Regular']
df_reg.head()

,Item,,Designer,Category,Item Name,Price,Qty Avail,Packaging,UPC/EAN,Case Count,Order Qty,Gender
0,12672,1.0,Abercrombie & Fitch,Regular,Abercrombie & Fitch Fierce 1.7 EDC Sp Men,46.5,56,New In Box,634349765,12,,Men
1,26413,1.0,Al Haramain,Regular,Al Haramain Amber Oud Tobacco Edition 2.0 EDP ...,38.5,more than 360,New In Box,6291100132171,- None -,,Men
2,10689,1.0,Alfred Sung,Regular,Alfred Sung Hei 3.4 EDT Sp Men,12.5,more than 360,New In Box,067724200017,24,,Men
3,13377,1.0,Alfred Sung,Regular,Alfred Sung Shi 3.4 EDP Sp Women,14.5,12,New In Box,067724271116,24,,Women
4,11668,1.0,Animale,Regular,Animale 3.4 EDP Sp Women,21.5,more than 360,New In Box,892456000037,24,,Women


In [13]:
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 781 entries, 0 to 780
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Item        781 non-null    int64  
 1               781 non-null    float64
 2   Designer    781 non-null    object 
 3   Category    781 non-null    object 
 4   Item Name   781 non-null    object 
 5   Price       781 non-null    float64
 6   Qty Avail   781 non-null    object 
 7   Packaging   781 non-null    object 
 8   UPC/EAN     781 non-null    object 
 9   Case Count  781 non-null    object 
 10  Order Qty   781 non-null    object 
 11  Gender      781 non-null    object 
dtypes: float64(2), int64(1), object(9)
memory usage: 79.3+ KB


In [14]:
upc_arr = df_reg['UPC/EAN'].loc[df_reg['UPC/EAN'] != '- None -'].to_numpy()
upc_arr.size

779

In [38]:
# Selenium settings
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-agent=[user-agent string]")
chrome_options.add_argument('--disable-notifications')
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument("window-size=1920,1080")

driver_manager = ChromeDriverManager().install()

pairs_list = []
for upc in tqdm(upc_arr):
    
    # New instance of driver.
    driver = webdriver.Chrome(driver_manager,
                          options = chrome_options)

    # Get ingo from page and close the page
    url = make_url(upc)
    driver.get(url)
    soup = bs(driver.page_source, 'html.parser') 
    driver.quit()
    
    # Parse page for required data
    soups_list = soup.find_all('span', {'class':'a-offscreen'})
    soup_texts = [item.text for item in soups_list]
    prices = [clean_price(price) for price in soup_texts]
    
    # Contextualize data
    tup_pair = (upc,prices)
    pairs_list.append(tup_pair)




====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\Fardin\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
100%|██████████| 779/779 [1:34:53<00:00,  7.31s/it]  


In [39]:
pairs_list

[('634349765', []),
 ('6291100132171', ['211.30']),
 ('067724200017', ['259.99']),
 ('067724271116', ['173.56']),
 ('892456000037', ['259.00', '454.16', '145.90']),
 ('892456000518', ['149.20']),
 ('892456000310',
  ['136.27', '212.06', '201.67', '189.19', '125.00', '345.16', '112.70']),
 ('892456000525', ['206.26']),
 ('8411061923221', ['58.21', '37.99', '43.65']),
 ('022548006719', ['232.03', '99.00', '111.65', '79.00']),
 ('812256020301',
  ['188.44',
   '186.00',
   '227.03',
   '57.00',
   '299.00',
   '78.00',
   '299.00',
   '99.00',
   '79.00',
   '235.00',
   '175.00',
   '299.00',
   '225.00',
   '79.00',
   '45.00']),
 ('812256022480', []),
 ('812256025467', []),
 ('812256021711', ['169.90']),
 ('812256024279',
  ['189.99',
   '201.15',
   '299.00',
   '299.00',
   '30.00',
   '79.00',
   '175.00',
   '299.00',
   '225.00',
   '225.00',
   '79.00',
   '135.00',
   '299.00']),
 ('3351500920037', ['167.58', '124.25', '108.37', '120.25']),
 ('3351500020409', ['124.25', '108.37'

In [43]:
pairs_arr = np.array(pairs_list, dtype = object)
pairs_arr

array([['634349765', list([])],
       ['6291100132171', list(['211.30'])],
       ['067724200017', list(['259.99'])],
       ...,
       ['3365440002173', list(['285.00'])],
       ['3365440002197', list(['361.46', '349.00', '446.57'])],
       ['3614271716026', list(['240.00'])]], dtype=object)

In [45]:
with open('scraped.npy', 'wb') as f:
    np.save(f,pairs_arr)

### Continuing session later

In [16]:
pairs_arr = np.load('scraped.npy', allow_pickle = True)
pairs_arr

array([['634349765', list([])],
       ['6291100132171', list(['211.30'])],
       ['067724200017', list(['259.99'])],
       ...,
       ['3365440002173', list(['285.00'])],
       ['3365440002197', list(['361.46', '349.00', '446.57'])],
       ['3614271716026', list(['240.00'])]], dtype=object)

In [24]:
## Carrying off where We last left it
mask = df['Category'].isin(['Gift Set', 'Independent Brand', 'Body Product'])
df_g = df[mask]
df_g.head()

,Item,,Designer,Category,Item Name,Price,Qty Avail,Packaging,UPC/EAN,Case Count,Order Qty,Gender
781,26420,2.0,Ariana Grande,Gift Set,Ariana Grande Cloud 3.4 EDP 3PC Gift Set,48.0,more than 360,New In Box,812256027737,6,,Women
782,27479,2.0,Ariana Grande,Gift Set,Ariana Grande Thank U Next 3.4 EDP 3pc Women Set,44.0,146,New In Box,812256027720,6,,Women
783,28302,2.0,Armaf,Gift Set,Armaf Hunter Intense 4PC Men Set,23.5,2,New In Box,6294015136821,12,,Men
784,29311,2.0,Azzaro,Gift Set,Azzaro Chrome 3PC Men Set,41.0,145,New In Box,3351500017003,3,,Men
785,26811,2.0,Azzaro,Gift Set,Azzaro Wanted 3.4 EDT Sp + 3.4 Hair & Body Sha...,35.5,2,New In Box,3351500010813,3,,Men


In [25]:
df_g['Category'].unique()

array(['Gift Set', 'Independent Brand', 'Body Product'], dtype=object)

In [26]:
df_g.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 781 to 1216
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Item        250 non-null    int64  
 1               250 non-null    float64
 2   Designer    250 non-null    object 
 3   Category    250 non-null    object 
 4   Item Name   250 non-null    object 
 5   Price       250 non-null    float64
 6   Qty Avail   250 non-null    object 
 7   Packaging   250 non-null    object 
 8   UPC/EAN     250 non-null    object 
 9   Case Count  250 non-null    object 
 10  Order Qty   250 non-null    object 
 11  Gender      250 non-null    object 
dtypes: float64(2), int64(1), object(9)
memory usage: 25.4+ KB


In [28]:
upc_arrg = df_g['UPC/EAN'].loc[df_g['UPC/EAN'] != '- None -'].to_numpy()
upc_arrg.size

250

In [29]:
pairs_listg = []
for upc in tqdm(upc_arrg):
    
    # New instance of driver.
    driver = webdriver.Chrome(driver_manager,
                          options = chrome_options)

    # Get ingo from page and close the page
    url = make_url(upc)
    driver.get(url)
    soup = bs(driver.page_source, 'html.parser') 
    driver.quit()
    
    # Parse page for required data
    soups_list = soup.find_all('span', {'class':'a-offscreen'})
    soup_texts = [item.text for item in soups_list]
    prices = [clean_price(price) for price in soup_texts]
    
    # Contextualize data
    tup_pair = (upc,prices)
    pairs_listg.append(tup_pair)

100%|██████████| 250/250 [26:20<00:00,  6.32s/it]


In [33]:
# Save it too
with open('scraped2.npy', 'wb') as f:
    np.save(f,np.array(pairs_listg, dtype = object))

In [39]:
final_arr = np.vstack([pairs_arr,np.array(pairs_listg, dtype = object)])
final_arr

array([['634349765', list([])],
       ['6291100132171', list(['211.30'])],
       ['067724200017', list(['259.99'])],
       ...,
       ['0667548099257', list(['64.50'])],
       ['0667548099288', list(['42.94', '130.60'])],
       ['031655063102', list(['139.90'])]], dtype=object)

In [48]:
flist = final_arr.tolist()
flist[0:5]

[['634349765', []],
 ['6291100132171', ['211.30']],
 ['067724200017', ['259.99']],
 ['067724271116', ['173.56']],
 ['892456000037', ['259.00', '454.16', '145.90']]]

In [51]:
processed = []

for upc,prices in flist:
    if not prices:
        continue
    elif len(prices) == 1:
        processed.append((upc,float(prices[0])))
    else:
        processed.append((upc,float(min(prices))))
        
processed[0:5]

[('6291100132171', 211.3),
 ('067724200017', 259.99),
 ('067724271116', 173.56),
 ('892456000037', 145.9),
 ('892456000518', 149.2)]

In [76]:
idx = [x[0] for x in processed]
vals = [x[1] for x in processed]

col = pd.Series(vals, index = idx, name = 'scraped_price')
col = col*0.27
col

6291100132171    57.0510
067724200017     70.1973
067724271116     46.8612
892456000037     39.3930
892456000518     40.2840
                  ...   
3607340373770    43.9506
3439600056259    89.1000
0667548099257    17.4150
0667548099288    35.2620
031655063102     37.7730
Name: scraped_price, Length: 773, dtype: float64

In [84]:
df = df.join(col, on = 'UPC/EAN')
df

,Item,,Designer,Category,Item Name,Price,Qty Avail,Packaging,UPC/EAN,Case Count,Order Qty,Gender,scraped_price
0,12672,1.0,Abercrombie & Fitch,Regular,Abercrombie & Fitch Fierce 1.7 EDC Sp Men,46.5,56,New In Box,634349765,12,,Men,NaN
1,26413,1.0,Al Haramain,Regular,Al Haramain Amber Oud Tobacco Edition 2.0 EDP ...,38.5,more than 360,New In Box,6291100132171,- None -,,Men,57.0510
2,10689,1.0,Alfred Sung,Regular,Alfred Sung Hei 3.4 EDT Sp Men,12.5,more than 360,New In Box,067724200017,24,,Men,70.1973
3,13377,1.0,Alfred Sung,Regular,Alfred Sung Shi 3.4 EDP Sp Women,14.5,12,New In Box,067724271116,24,,Women,46.8612
4,11668,1.0,Animale,Regular,Animale 3.4 EDP Sp Women,21.5,more than 360,New In Box,892456000037,24,,Women,39.3930
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,28529,NaN,Bond 9,HBA,Bond No. 9 Madison Square Park 6.4 Scented Candle,55.0,10,New In Box,888874002333,6,,Unisex,NaN
1285,28531,NaN,Bond 9,HBA,Bond No. 9 Nolita 6.4 Scented Candle,60.0,4,New In Box,888874005747,6,,Unisex,NaN
1286,28533,NaN,Bond 9,HBA,Bond No. 9 The Scent Of Peace 6.4 Scented Candle,55.0,6,New In Box,888874001541,2,,Unisex,NaN
1287,26724,NaN,Nuit D'Orient,HBA,Nuit D'Orient Hand Sanitizer 16.0 OZ. Pump,3.0,15,Unboxed,860002058931,24,,Unisex,NaN


In [85]:
df['price_difference'] = df['scraped_price']  - df['Price']
df.head()

,Item,,Designer,Category,Item Name,Price,Qty Avail,Packaging,UPC/EAN,Case Count,Order Qty,Gender,scraped_price,price_difference
0,12672,1.0,Abercrombie & Fitch,Regular,Abercrombie & Fitch Fierce 1.7 EDC Sp Men,46.5,56,New In Box,634349765,12,,Men,NaN,NaN
1,26413,1.0,Al Haramain,Regular,Al Haramain Amber Oud Tobacco Edition 2.0 EDP ...,38.5,more than 360,New In Box,6291100132171,- None -,,Men,57.0510,18.5510
2,10689,1.0,Alfred Sung,Regular,Alfred Sung Hei 3.4 EDT Sp Men,12.5,more than 360,New In Box,067724200017,24,,Men,70.1973,57.6973
3,13377,1.0,Alfred Sung,Regular,Alfred Sung Shi 3.4 EDP Sp Women,14.5,12,New In Box,067724271116,24,,Women,46.8612,32.3612
4,11668,1.0,Animale,Regular,Animale 3.4 EDP Sp Women,21.5,more than 360,New In Box,892456000037,24,,Women,39.3930,17.8930


In [102]:
conditions  = [np.isnan(df['price_difference']),\
              (df['price_difference'] > 0),\
              (df['price_difference'] <= 0)]
choices     = [np.nan, 'HIGHER', 'LOWER' ]
df['qualitative'] = np.select(conditions, choices)
    
df.head()

,Item,,Designer,Category,Item Name,Price,Qty Avail,Packaging,UPC/EAN,Case Count,Order Qty,Gender,scraped_price,price_difference,qualitative
0,12672,1.0,Abercrombie & Fitch,Regular,Abercrombie & Fitch Fierce 1.7 EDC Sp Men,46.5,56,New In Box,634349765,12,,Men,NaN,NaN,nan
1,26413,1.0,Al Haramain,Regular,Al Haramain Amber Oud Tobacco Edition 2.0 EDP ...,38.5,more than 360,New In Box,6291100132171,- None -,,Men,57.0510,18.5510,HIGHER
2,10689,1.0,Alfred Sung,Regular,Alfred Sung Hei 3.4 EDT Sp Men,12.5,more than 360,New In Box,067724200017,24,,Men,70.1973,57.6973,HIGHER
3,13377,1.0,Alfred Sung,Regular,Alfred Sung Shi 3.4 EDP Sp Women,14.5,12,New In Box,067724271116,24,,Women,46.8612,32.3612,HIGHER
4,11668,1.0,Animale,Regular,Animale 3.4 EDP Sp Women,21.5,more than 360,New In Box,892456000037,24,,Women,39.3930,17.8930,HIGHER


In [105]:
df['urls'] = df['UPC/EAN'].apply(lambda x: make_url(x))
df.head()

,Item,,Designer,Category,Item Name,Price,Qty Avail,Packaging,UPC/EAN,Case Count,Order Qty,Gender,scraped_price,price_difference,qualitative,urls
0,12672,1.0,Abercrombie & Fitch,Regular,Abercrombie & Fitch Fierce 1.7 EDC Sp Men,46.5,56,New In Box,634349765,12,,Men,NaN,NaN,nan,https://www.amazon.ae/s?k=634349765
1,26413,1.0,Al Haramain,Regular,Al Haramain Amber Oud Tobacco Edition 2.0 EDP ...,38.5,more than 360,New In Box,6291100132171,- None -,,Men,57.0510,18.5510,HIGHER,https://www.amazon.ae/s?k=6291100132171
2,10689,1.0,Alfred Sung,Regular,Alfred Sung Hei 3.4 EDT Sp Men,12.5,more than 360,New In Box,067724200017,24,,Men,70.1973,57.6973,HIGHER,https://www.amazon.ae/s?k=067724200017
3,13377,1.0,Alfred Sung,Regular,Alfred Sung Shi 3.4 EDP Sp Women,14.5,12,New In Box,067724271116,24,,Women,46.8612,32.3612,HIGHER,https://www.amazon.ae/s?k=067724271116
4,11668,1.0,Animale,Regular,Animale 3.4 EDP Sp Women,21.5,more than 360,New In Box,892456000037,24,,Women,39.3930,17.8930,HIGHER,https://www.amazon.ae/s?k=892456000037


In [106]:
df.to_excel('prices_joined.xlsx', index = False)

In [108]:
df_l = df.loc[df['qualitative'] == 'LOWER']
df_l.head()

,Item,,Designer,Category,Item Name,Price,Qty Avail,Packaging,UPC/EAN,Case Count,Order Qty,Gender,scraped_price,price_difference,qualitative,urls
14,26530,1.0,Ariana Grande,Regular,Ariana Grande Thank U Next 3.4 EDP Sp Women,36.5,3,New In Box,812256024279,24,,Women,36.4500,-0.0500,LOWER,https://www.amazon.ae/s?k=812256024279
38,23195,1.0,Bond 9,Regular,Bond No. 9 Bleecker Street 3.3 EDP Sp Unisex,220.0,49,New In Box,888874001466,3,,Unisex,112.2768,-107.7232,LOWER,https://www.amazon.ae/s?k=888874001466
41,23180,1.0,Bond 9,Regular,Bond No. 9 Central Park West 1.7 EDP Sp Unisex,100.0,22,New In Box,888874002418,3,,Unisex,27.8100,-72.1900,LOWER,https://www.amazon.ae/s?k=888874002418
42,27845,1.0,Bond 9,Regular,Bond No. 9 Dubai Black Sapphire 3.3 EDP Sp Unisex,205.0,1,New In Box,888874005631,3,,Unisex,184.9500,-20.0500,LOWER,https://www.amazon.ae/s?k=888874005631
44,26349,1.0,Bond 9,Regular,Bond No. 9 Gold Coast 3.4 EDP Sp Women,155.0,40,New In Box,888874006829,3,,Women,69.8463,-85.1537,LOWER,https://www.amazon.ae/s?k=888874006829


In [109]:
df_l.to_excel('lower.xlsx', index = False)